# perceptron

### inputs 
 - training data
 - labels
 - rate of learning
 - epochs
 - random seed

### outputs 
 - final weights
 - no. of epochs
 - error(loss?) in each epoch



In [18]:
import pandas as pd

df = pd.read_csv("datasets/train_dataset1.csv")

print(df)

           x1        x2  label
0    0.254018 -0.402391     -1
1    0.180883  0.510887      1
2    0.227152  0.467715      1
3   -0.122832  0.502635      1
4    0.437193 -0.579592     -1
..        ...       ...    ...
135  0.652045  0.575919      1
136 -0.001194  0.500983      1
137  0.387381  0.714670      1
138  1.155703  0.502584     -1
139  0.783104  0.641614      1

[140 rows x 3 columns]


In [32]:
import numpy as np

inputs = df.drop(['label'], axis=1)
train_output = df[['label']]
weights = [0.3, 0.4]

print(inputs[['x1']])
print(train_output)

def activation(weighted_sum):
    if weighted_sum > thresh:
        return 1
    else:
        return 0

def perceptron_one_neuron():
    weighted_sum=0
    
    for x,w in zip(inputs[['x1']], weights):
        weighted_sum += (w*x)

    final = activation(weighted_sum)

    return final

output = perceptron_one_neuron()

           x1
0    0.254018
1    0.180883
2    0.227152
3   -0.122832
4    0.437193
..        ...
135  0.652045
136 -0.001194
137  0.387381
138  1.155703
139  0.783104

[140 rows x 1 columns]
     label
0       -1
1        1
2        1
3        1
4       -1
..     ...
135      1
136      1
137      1
138     -1
139      1

[140 rows x 1 columns]


TypeError: can't multiply sequence by non-int of type 'float'